Script to optimize the fluorescence from single atom imaging.\
We use M-LOOP to feedback the information between the experiment and this computer, which controls the parameters to optimize.\
The input parameters are produced by M-LOOP in the file "exp_input.txt". This file is read by this script and the parameters passed as variables. Then the program is sent to OPX for execution: it waits the trigger from Logicbox. Then an image is produced, saved in "Whitlock-PC", has to be passed to this computer. Once it's in this computer, we evaluate the cost and produce the file "exp_output" which is read by M-LOOP: it applies the opimization algorithm specified in the configuration file and produces a new "exp_input" file. The loop is closed.

So we have two files that work together: one for control and a second one that reads the received image and evaluates the cost.

In [73]:
from qm.qua import *
from qm import QuantumMachinesManager, SimulationConfig
from quam.components import BasicQuAM, SingleChannel
from quam.components.pulses import SquarePulse
from qualang_tools.units import unit
from QuAM_utilities import *

import os
import matplotlib.pyplot as plt
from matplotlib import patches
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler, PatternMatchingEventHandler
import numpy as np
from numpy import array
from astropy.io import fits

u = unit() 


In [74]:
# silence the warnings?
import warnings
warnings.filterwarnings("ignore")

from IPython.display import clear_output

In [75]:
ip_address = '130.79.148.167'
qmm = QuantumMachinesManager(host=ip_address)

2024-11-20 18:53:48,051 - qm - INFO     - Performing health check
2024-11-20 18:53:48,098 - qm - INFO     - Health check passed


Configure the machine: define the channels and associate the pulses

In [76]:
machine = BasicQuAM()

# AOM RFs
cool_double_rf = 200*u.MHz
repump_double_rf = 195*u.MHz

# TWEEZER LOADING
tweez_central_freq = 75.1*u.MHz
tweez_difference_freq = 15*u.MHz
tweez_low_freq = tweez_central_freq - tweez_difference_freq/2
tweez_high_freq = tweez_central_freq + tweez_difference_freq/2

tweez_ramp_amplitude = 0.07
tweez_ramp_time = 100*u.ms #50*u.us #
tweez_plateau_amplitude = tweez_ramp_amplitude
tweez_plateau_time = 150.15*u.ms #50*u.us #

# GAP 1
gap1_time = 0.6*u.ms #200*u.ns#

# COLLISION
coll_time = 80*u.ms #100*u.us #
tweez_square_amplitude_coll = 0.15
repump_square_amplitude_coll_double = 0.38 # fixed
cool_square_amplitude_coll_double = 0.38 # fixed
coll_modulation_frequency = 1.7*u.MHz
imag_modulation_frequency = 2.5*u.MHz
tweez_duty_cycle = 0.4 # high fraction for tweezer
rc_duty_cycle = 0.3 # high fraction for repumper and cooler
rc_delay_fraction = 0.05 # delay for repumper and cooler (fraction of period) 
imag_rc_delay_fraction = 0.07 # delay for repumper and cooler during imaging (fraction of period) 
# the sum of the repumper/cooler imag_rc_delay_fraction + rc_duty_cycle duty  must be less than 1
# otherwize, imag_rc_low_time = 0 (min value 16 ns)

# GAP 2
gap2_time = 5*u.ms #20*u.us #
# look if after this we have an intrinsic delay of 35 ns?

# IMAGING
# the duty cycle parameters are the same as for collision
imag_time = 80*u.ms #80*u.us #
tweez_square_amplitude_imag = 0.15
repump_square_amplitude_imag_double = 0.38 # fixed
cool_square_amplitude_imag_double = 0.38 # fixed

#coll_time_single_pass = gap1_time + coll_time
#imag_time_single_pass = gap2_time + imag_time

# DELAY FOR IMAGING
delay_imaging_time = 200.15*u.ms #20*u.us #

# FOR CONTINUOUSLY DRIVEN AOMs: cooler/repumper single pass & two spots tweezers
# this is for amplitude modulation
continuous_time = 50*u.ms

# collision
repump_square_amplitude_coll_single = 0.01
cool_square_amplitude_coll_single = 0.04

# imaging
repump_square_amplitude_imag_single = 0.01
cool_square_amplitude_imag_single = 0.05

spot_amplitude1 = 0.38 # fixed
spot_amplitude2 = 0.38 # fixed

In [77]:
## create all the time staps for imaging modulation frequency
imag_modulation_period = 1/imag_modulation_frequency
imag_tweez_high_time = 1/imag_modulation_frequency*tweez_duty_cycle
imag_tweez_low_time = 1/imag_modulation_frequency*(1-tweez_duty_cycle)
imag_rc_high_time = 1/imag_modulation_frequency*rc_duty_cycle
imag_rc_delay_time = 1/imag_modulation_frequency*imag_rc_delay_fraction
imag_rc_low_time = imag_modulation_period-imag_rc_delay_time-imag_rc_high_time
number_of_periods_imaging = round(imag_time/(imag_modulation_period*u.s)) # used as argument

# for debugging
print("Number of imaging periods:", number_of_periods_imaging, "\n")
print(f"Low duration for tweezer: {round(imag_tweez_low_time*u.s/4)*4}ns")
print(f"High duration for tweezer: {round(imag_tweez_high_time*u.s/4)*4}ns")
print(f"Total: {round(imag_tweez_low_time*1e9/4)*4 + round(imag_tweez_high_time*u.s/4)*4}ns \n")
print(f"Delay duration for repumper/cooler: {round(imag_rc_delay_time*u.s/4)*4}ns")
print(f"High duration for repumper/cooler: {round(imag_rc_high_time*u.s/4)*4}ns")
print(f"Low duration for repumper/cooler: {round(imag_rc_low_time*u.s/4)*4}ns")
print(f"Total: {round(imag_rc_delay_time*u.s/4)*4 + round(imag_rc_low_time*u.s/4)*4 + round(imag_rc_high_time*u.s/4)*4}ns \n")

Number of imaging periods: 200000 

Low duration for tweezer: 240ns
High duration for tweezer: 160ns
Total: 400ns 

Delay duration for repumper/cooler: 28ns
High duration for repumper/cooler: 120ns
Low duration for repumper/cooler: 252ns
Total: 400ns 



In [78]:
# additional dependent variables useful for pulse definition
# NB: the time durations must be multiples of the clock cycle!

# these are not used in the function, but are printed for debug
tweez_ramp_step_time = 40*u.ns
n_steps = round(tweez_ramp_time/tweez_ramp_step_time)
plateau_intermediate_duration = 10*u.us # prone to bug: this must be an integer of plateau_duration # INSIDE THE FUNCTION, AS AN ARGUMENT?
plateau_num_portions = round(tweez_plateau_time/plateau_intermediate_duration) #INSIDE THE FUNCTION

# these are in seconds, multiply by u.s before passing as arguments
coll_modulation_period = 1/coll_modulation_frequency
tweez_high_time = 1/coll_modulation_frequency*tweez_duty_cycle
tweez_low_time = 1/coll_modulation_frequency*(1-tweez_duty_cycle)
rc_high_time = 1/coll_modulation_frequency*rc_duty_cycle
rc_delay_time = 1/coll_modulation_frequency*rc_delay_fraction
rc_low_time = coll_modulation_period-rc_delay_time-rc_high_time
number_of_periods_collision = round(coll_time/(coll_modulation_period*u.s)) # used as argument


# for debugging
print(f"\n Number of steps for the tweezer ramp: {n_steps}, each of {tweez_ramp_step_time} ns")
print(f"Number of plateau portions: {plateau_num_portions}")
print("Number of collision periods:", number_of_periods_collision)
print(f"Low duration for tweezer: {round(tweez_low_time*u.s/4)*4}ns")
print(f"High duration for tweezer: {round(tweez_high_time*u.s/4)*4}ns")
print(f"Total: {round(tweez_low_time*1e9/4)*4 + round(tweez_high_time*u.s/4)*4}ns \n")
print(f"Delay duration for repumper/cooler: {round(rc_delay_time*u.s/4)*4}ns")
print(f"High duration for repumper/cooler: {round(rc_high_time*u.s/4)*4}ns")
print(f"Low duration for repumper/cooler: {round(rc_low_time*u.s/4)*4}ns")
print(f"Total: {round(rc_delay_time*u.s/4)*4 + round(rc_low_time*u.s/4)*4 + round(rc_high_time*u.s/4)*4}ns \n")

# add a warning if the durations are not the same


 Number of steps for the tweezer ramp: 2500000, each of 40.0 ns
Number of plateau portions: 15015
Number of collision periods: 136000
Low duration for tweezer: 352ns
High duration for tweezer: 236ns
Total: 588ns 

Delay duration for repumper/cooler: 28ns
High duration for repumper/cooler: 176ns
Low duration for repumper/cooler: 384ns
Total: 588ns 



Configure the machine: define the channels and add the pulses

In [79]:
machine.channels['AOM_single_repumper'] = AOM_single_repumper = SingleChannel(opx_output=('con1', 1), intermediate_frequency=200e6)
machine.channels['AOM_double_repumper'] = AOM_double_repumper = SingleChannel(opx_output=('con1', 2), intermediate_frequency=repump_double_rf)
machine.channels['AOM_single_cooler'] = AOM_single_cooler = SingleChannel(opx_output=('con1', 3), intermediate_frequency=200e6)
machine.channels['AOM_double_cooler'] = AOM_double_cooler = SingleChannel(opx_output=('con1', 4), intermediate_frequency=cool_double_rf)
machine.channels['AOM_tweez_mod'] = AOM_tweez_mod = SingleChannel(opx_output=('con1', 5), intermediate_frequency=89.161e6) # for tweezer modulation
machine.channels['AOM_tweez_spots1'] = AOM_tweez_spots1 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=tweez_high_freq) # first component for generating two spots
machine.channels['AOM_tweez_spots2'] = AOM_tweez_spots2 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=tweez_low_freq) # second component for generating two spots

AOM_single_repumper.operations['repump_continuous_coll'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=repump_square_amplitude_coll_single)
AOM_single_repumper.operations['repump_continuous_imag'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=repump_square_amplitude_imag_single)
AOM_single_cooler.operations['cool_continuous_coll'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=cool_square_amplitude_coll_single)
AOM_single_cooler.operations['cool_continuous_imag'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=cool_square_amplitude_imag_single)

AOM_double_repumper.operations['repump_high_pulse_coll'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=repump_square_amplitude_coll_double)
AOM_double_repumper.operations['repump_low_pulse_coll'] = SquarePulse(length=round(rc_low_time*u.s/4)*4, amplitude=0)
AOM_double_repumper.operations['repump_high_pulse_imag'] = SquarePulse(length=round(imag_rc_high_time*u.s/4)*4, amplitude=repump_square_amplitude_imag_double)
AOM_double_repumper.operations['repump_low_pulse_imag'] = repump_low_pulse_imag = SquarePulse(length=round(imag_rc_low_time*u.s/4)*4, amplitude=repump_square_amplitude_imag_double)


AOM_double_cooler.operations['cool_high_pulse_coll'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=cool_square_amplitude_coll_double)
AOM_double_cooler.operations['cool_low_pulse_coll']  = SquarePulse(length=round(rc_low_time*u.s/4)*4, amplitude=0)
AOM_double_cooler.operations['cool_high_pulse_imag'] = SquarePulse(length=round(imag_rc_high_time*u.s/4)*4, amplitude=cool_square_amplitude_imag_double)
AOM_double_cooler.operations['cool_low_pulse_imag'] = cool_low_pulse_imag = SquarePulse(length=round(imag_rc_low_time*u.s/4)*4, amplitude=0)


AOM_tweez_mod.operations['tweez_step_pulse'] = tweez_step_pulse = SquarePulse(length=round(tweez_ramp_step_time/4)*4, amplitude=tweez_ramp_amplitude) 
AOM_tweez_mod.operations['tweez_plateau_pulse'] = SquarePulse(length=100, amplitude=tweez_plateau_amplitude) # this value is overridden by the time/slice value
AOM_tweez_mod.operations['tweez_high_pulse_coll'] = SquarePulse(length=round(tweez_high_time*u.s/4)*4, amplitude=tweez_square_amplitude_coll)
AOM_tweez_mod.operations['tweez_high_pulse_imag'] = SquarePulse(length=round(imag_tweez_high_time*u.s/4)*4, amplitude=tweez_square_amplitude_imag)
AOM_tweez_mod.operations['tweez_low_pulse_coll'] = SquarePulse(length=round(tweez_low_time*u.s/4)*4, amplitude=0)
AOM_tweez_mod.operations['tweez_low_pulse_imag'] = SquarePulse(length=round(imag_tweez_low_time*u.s/4)*4, amplitude=0)
AOM_tweez_spots1.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude1)
AOM_tweez_spots2.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude2)


In [80]:
AOM_double_cooler.operations['cool_high_pulse_imag'].length

120

In [81]:
qua_config = machine.generate_config()

Write the QUA program

Modify the utilities script in order to handle the time parameter?

In [82]:
"""with program() as strobo_collision_imaging:

    #update_frequency('AOM_double_cooler', 200e6)

    with infinite_loop_():

        AOM_tweez_spots1.play("tweez_continuous") # this is not compatible with an external trigger because has to be played continuously
        AOM_tweez_spots2.play("tweez_continuous")
        
    with infinite_loop_():
        #update_frequency('AOM_double_repumper', 200e6, keep_phase=True)
        # might work w/o string
        wait_for_trigger('AOM_tweez_mod') 
        wait_for_trigger('AOM_double_repumper')
        wait_for_trigger('AOM_double_cooler')
        wait_for_trigger('AOM_single_repumper')
        wait_for_trigger('AOM_single_cooler')

        # atoms loading
        long_ramp('tweez_step_pulse', 'AOM_tweez_mod', tweez_ramp_time)
        long_pulse('tweez_plateau_pulse', 'AOM_tweez_mod', tweez_plateau_time)
        align('AOM_tweez_mod', 'AOM_double_cooler', 'AOM_double_repumper','AOM_single_cooler','AOM_single_repumper')

        # gap1
        play('tweez_step_pulse', 'AOM_tweez_mod', duration=round(gap1_time/4))
        play('cool_low_pulse_coll', 'AOM_double_cooler', duration=round(gap1_time/4))
        play('repump_low_pulse_coll', 'AOM_double_repumper', duration=round(gap1_time/4))
        
        #long_pulse('cool_continuous_coll', 'AOM_single_cooler', gap1_time)
        #long_pulse('repump_continuous_coll', 'AOM_single_repumper', gap1_time)

        long_pulse('cool_continuous_coll', 'AOM_single_cooler', gap1_time + coll_time) # try to change
        long_pulse('repump_continuous_coll', 'AOM_single_repumper', gap1_time + coll_time)

        # collision
        stroboscopic(('tweez_low_pulse_coll','tweez_high_pulse_coll'), 'AOM_tweez_mod', number_of_periods_collision)
        stroboscopic(('cool_high_pulse_coll','cool_low_pulse_coll'), 'AOM_double_cooler', number_of_periods_collision, rc_delay_time)
        stroboscopic(('repump_high_pulse_coll','repump_low_pulse_coll'), 'AOM_double_repumper', number_of_periods_collision, rc_delay_time)

        #update_frequency('AOM_double_cooler', my_frequency, keep_phase=True)
        #update_frequency('AOM_double_repumper', 195e6, keep_phase=True)

        # gap2
        play('tweez_high_pulse_imag', 'AOM_tweez_mod', duration=round(gap2_time/4))
        play('cool_low_pulse_coll', 'AOM_double_cooler', duration=round(gap2_time/4))
        play('repump_low_pulse_coll', 'AOM_double_repumper', duration=round(gap2_time/4))

        long_pulse('cool_continuous_imag', 'AOM_single_cooler', gap2_time + imag_time) # try to change
        long_pulse('repump_continuous_imag', 'AOM_single_repumper', gap2_time + imag_time)

        # imaging
        stroboscopic(('tweez_low_pulse_imag','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse_imag'), 'AOM_double_repumper', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('cool_high_pulse_imag','cool_low_pulse_imag'), 'AOM_double_cooler', number_of_periods_imaging, rc_delay_time)

        # delay for image
        long_pulse('tweez_high_pulse_imag', 'AOM_tweez_mod', delay_imaging_time)
        long_pulse('cool_low_pulse_imag', 'AOM_double_cooler', delay_imaging_time)
        long_pulse('repump_low_pulse_imag', 'AOM_double_repumper', delay_imaging_time)

        long_pulse('cool_continuous_imag', 'AOM_single_cooler', delay_imaging_time + imag_time) # try to change
        long_pulse('repump_continuous_imag', 'AOM_single_repumper', delay_imaging_time + imag_time)
        
        #align('AOM_tweez_mod', 'AOM_double_cooler', 'AOM_double_repumper', 'AOM_single_cooler', 'AOM_single_repumper')
        
        # imaging
        stroboscopic(('tweez_low_pulse_coll','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
        stroboscopic(('cool_high_pulse_imag','cool_low_pulse_imag'), 'AOM_double_cooler', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse_imag'), 'AOM_double_repumper', number_of_periods_imaging, rc_delay_time)
        """


'with program() as strobo_collision_imaging:\n\n    #update_frequency(\'AOM_double_cooler\', 200e6)\n\n    with infinite_loop_():\n\n        AOM_tweez_spots1.play("tweez_continuous") # this is not compatible with an external trigger because has to be played continuously\n        AOM_tweez_spots2.play("tweez_continuous")\n        \n    with infinite_loop_():\n        #update_frequency(\'AOM_double_repumper\', 200e6, keep_phase=True)\n        # might work w/o string\n        wait_for_trigger(\'AOM_tweez_mod\') \n        wait_for_trigger(\'AOM_double_repumper\')\n        wait_for_trigger(\'AOM_double_cooler\')\n        wait_for_trigger(\'AOM_single_repumper\')\n        wait_for_trigger(\'AOM_single_cooler\')\n\n        # atoms loading\n        long_ramp(\'tweez_step_pulse\', \'AOM_tweez_mod\', tweez_ramp_time)\n        long_pulse(\'tweez_plateau_pulse\', \'AOM_tweez_mod\', tweez_plateau_time)\n        align(\'AOM_tweez_mod\', \'AOM_double_cooler\', \'AOM_double_repumper\',\'AOM_single_coo

In [83]:
class InputEventHandler(PatternMatchingEventHandler):
    def __init__(self, patterns, params):
        super().__init__(patterns=patterns, ignore_directories=True, case_sensitive=False) # idk why but works only if case sensitive is False (default)
        self.params_to_update = params #it's a reference, ie changes the operation itself

    def on_created(self, event):
        # this functoin is called whenever a new "pattern" file is created
        
        clear_output(wait=True)
        print(f"Processing file: {event.src_path}")
        time.sleep(0.5) # wait for M-LOOP to write the file
        with open(event.src_path, 'r') as file:
            line = file.readline().strip()
            new_parameters = np.array(eval(line.split('=')[1].strip())) # take the argument after the "=" and evaluate the string as python variable
        os.remove(event.src_path)
        print("The Dog has watched:", new_parameters) # for debugging

        imag_rc_delay_fraction = new_parameters[0]
        imag_rc_delay_time = 1/imag_modulation_frequency*imag_rc_delay_fraction
        imag_rc_low_time = imag_modulation_period-imag_rc_delay_time-imag_rc_high_time
        print(imag_rc_low_time)
        
        # QUA PROGRAM
        with program() as delay_optimization:

                AOM_tweez_spots1.play("tweez_continuous") # this is not compatible with an external trigger because has to be played continuously
                AOM_tweez_spots2.play("tweez_continuous")
        
                #update_frequency('AOM_double_repumper', 200e6, keep_phase=True)
                # might work w/o string
                wait_for_trigger('AOM_tweez_mod') 
                wait_for_trigger('AOM_double_repumper')
                wait_for_trigger('AOM_double_cooler')
                wait_for_trigger('AOM_single_repumper')
                wait_for_trigger('AOM_single_cooler')

                # atoms loading
                long_ramp('tweez_step_pulse', 'AOM_tweez_mod', tweez_ramp_time)
                long_pulse('tweez_plateau_pulse', 'AOM_tweez_mod', tweez_plateau_time)
                align('AOM_tweez_mod', 'AOM_double_cooler', 'AOM_double_repumper','AOM_single_cooler','AOM_single_repumper')

                # gap1
                play('tweez_step_pulse', 'AOM_tweez_mod', duration=round(gap1_time/4))
                play('cool_low_pulse_coll', 'AOM_double_cooler', duration=round(gap1_time/4))
                play('repump_low_pulse_coll', 'AOM_double_repumper', duration=round(gap1_time/4))
                
                #long_pulse('cool_continuous_coll', 'AOM_single_cooler', gap1_time)
                #long_pulse('repump_continuous_coll', 'AOM_single_repumper', gap1_time)

                long_pulse('cool_continuous_coll', 'AOM_single_cooler', gap1_time + coll_time) # try to change
                long_pulse('repump_continuous_coll', 'AOM_single_repumper', gap1_time + coll_time)

                # collision
                stroboscopic(('tweez_low_pulse_coll','tweez_high_pulse_coll'), 'AOM_tweez_mod', number_of_periods_collision)
                stroboscopic(('cool_high_pulse_coll','cool_low_pulse_coll'), 'AOM_double_cooler', number_of_periods_collision, rc_delay_time)
                stroboscopic(('repump_high_pulse_coll','repump_low_pulse_coll'), 'AOM_double_repumper', number_of_periods_collision, rc_delay_time)

                # gap2
                play('tweez_high_pulse_imag', 'AOM_tweez_mod', duration=round(gap2_time/4))
                play('cool_low_pulse_coll', 'AOM_double_cooler', duration=round(gap2_time/4))
                play('repump_low_pulse_coll', 'AOM_double_repumper', duration=round(gap2_time/4))

                long_pulse('cool_continuous_imag', 'AOM_single_cooler', gap2_time + imag_time) # try to change
                long_pulse('repump_continuous_imag', 'AOM_single_repumper', gap2_time + imag_time)

                # imaging
                stroboscopic(('tweez_low_pulse_imag','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
                stroboscopic(('repump_high_pulse_imag','repump_low_pulse_imag'), 'AOM_double_repumper', number_of_periods_imaging, imag_rc_delay_time)
                stroboscopic(('cool_high_pulse_imag','cool_low_pulse_imag'), 'AOM_double_cooler', number_of_periods_imaging, imag_rc_delay_time)

                # delay for image
                long_pulse('tweez_high_pulse_imag', 'AOM_tweez_mod', delay_imaging_time)
                long_pulse('cool_low_pulse_imag', 'AOM_double_cooler', delay_imaging_time)
                long_pulse('repump_low_pulse_imag', 'AOM_double_repumper', delay_imaging_time)

                long_pulse('cool_continuous_imag', 'AOM_single_cooler', delay_imaging_time + imag_time) # try to change
                long_pulse('repump_continuous_imag', 'AOM_single_repumper', delay_imaging_time + imag_time)
                                
                # imaging
                stroboscopic(('tweez_low_pulse_coll','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
                stroboscopic(('cool_high_pulse_imag','cool_low_pulse_imag'), 'AOM_double_cooler', number_of_periods_imaging, imag_rc_delay_time)
                stroboscopic(('repump_high_pulse_imag','repump_low_pulse_imag'), 'AOM_double_repumper', number_of_periods_imaging, imag_rc_delay_time)

        self.params_to_update[0].length = round(imag_rc_low_time*u.s/4)*4
        self.params_to_update[1].length = round(imag_rc_low_time*u.s/4)*4
             
        qua_config = machine.generate_config()
        print(f"From configuration:")
        print(f" cooler low length: {qua_config['pulses']['AOM_double_cooler.cool_low_pulse_imag.pulse']['length']}")
        print(f" repumper low length: {qua_config['pulses']['AOM_double_repumper.repump_low_pulse_imag.pulse']['length']}")
        qm = qmm.open_qm(qua_config)
        qm.execute(delay_optimization)


In [84]:
# get the parameters from the exp_input file 
path_to_watch = r"C:\Users\EQM\Giovanni\OPXsetup\OPX_EQM\Optimization\imaging_parameters"
pattern = 'exp_input.txt'
#parameters_to_update = [AOM_double_cooler, AOM_double_repumper]
parameters_to_update = [cool_low_pulse_imag, repump_low_pulse_imag]

if __name__ == '__main__':
    observer = Observer()
    event_handler = InputEventHandler(patterns=[pattern], params=parameters_to_update)
    observer.schedule(event_handler, path=path_to_watch, recursive=True)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

Processing file: C:\Users\EQM\Giovanni\OPXsetup\OPX_EQM\Optimization\imaging_parameters\exp_input.txt
The Dog has watched: [0.06330627]
2.5467749199999996e-07
From configuration:
 cooler low length: 256
 repumper low length: 256
2024-11-20 18:56:11,190 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing Quantum Machines: 
2024-11-20 18:56:11,191 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1732125365325 is canceling running job 1730223363085.
2024-11-20 18:56:11,215 - qm - INFO     - Sending program to QOP for compilation
2024-11-20 18:56:15,538 - qm - INFO     - Executing program


In [94]:
%matplotlib qt

Simulation = False

if Simulation == True:

    start_time = time.time()
    simulated_job = qmm.simulate(qua_config, strobo_collision_imaging, SimulationConfig(duration=int(400*1e3/4)))
    end_time = time.time()
    print(f"Simulation time: {end_time - start_time} s")

    # get DAC and digital samples
    samples = simulated_job.get_simulated_samples()
    plt.figure()
    plt.plot(samples.con1.analog["1"], "--", label="sg_repumper")
    plt.plot(samples.con1.analog["2"], "-", label="db_repumper")
    #plt.plot(samples.con1.analog["3"], "--", label="sg_cooler")
    #plt.plot(samples.con1.analog["4"], "-", label="db_cooler")
    plt.plot(samples.con1.analog["5"], "-", label="tweezer_mod")
    #plt.plot(samples.con1.analog["6"], "-", label="tweezer_beat")
    plt.legend()
    plt.xlabel("Time [ns]")
    plt.ylabel("Signal [V]")        
    plt.show()
else:
    qm = qmm.open_qm(qua_config)
    job = qm.execute(strobo_collision_imaging)

2024-11-14 18:55:39,151 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing Quantum Machines: 
2024-11-14 18:55:39,152 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1731606832957 is canceling running job 1730223360900.
2024-11-14 18:55:39,184 - qm - INFO     - Sending program to QOP for compilation
2024-11-14 18:55:39,535 - qm - INFO     - Executing program


In [96]:
job.halt()

True

### Ratatuja

In [26]:
repump_continuous_imag.amplitude = 29

In [27]:
qua_config = machine.generate_config()
qua_config

{'version': 1,
 'controllers': {'con1': {'analog_outputs': {1: {'delay': 0,
     'shareable': False,
     'offset': 0.0},
    2: {'delay': 0, 'shareable': False, 'offset': 0.0},
    3: {'delay': 0, 'shareable': False, 'offset': 0.0},
    4: {'delay': 0, 'shareable': False, 'offset': 0.0},
    5: {'delay': 0, 'shareable': False, 'offset': 0.0},
    6: {'delay': 0, 'shareable': False, 'offset': 0.0}}}},
 'elements': {'AOM_single_repumper': {'operations': {'repump_continuous_coll': 'AOM_single_repumper.repump_continuous_coll.pulse',
    'repump_continuous_imag': 'AOM_single_repumper.repump_continuous_imag.pulse'},
   'intermediate_frequency': 200000000.0,
   'singleInput': {'port': ('con1', 1)}},
  'AOM_double_repumper': {'operations': {'repump_high_pulse_coll': 'AOM_double_repumper.repump_high_pulse_coll.pulse',
    'repump_high_pulse_imag': 'AOM_double_repumper.repump_high_pulse_imag.pulse',
    'repump_low_pulse': 'AOM_double_repumper.repump_low_pulse.pulse'},
   'intermediate_frequenc